In [1]:
import torch
import pandas as pd
from torchvision import models, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os

In [2]:
import zipfile
test_zip_path = '/kaggle/input/dogs-vs-cats/test1.zip'
extract_to_directory = '/kaggle/working/'
with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

In [3]:


# Load the trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 2)  # Modify the last layer for 2 classes
model.load_state_dict(torch.load('/kaggle/input/notebook12c651d9a1/cats_vs_dogs_resnet18.pth'))  # Load your trained model
model = model.to(device)
model.eval()

# Define the image transformations (same as during training)
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Path to the test images directory
test_dir = '/kaggle/working/test1'  # Modify this with your test directory path

# List all image files in the test directory
test_images = os.listdir(test_dir)

# Prepare the list for storing the results
results = []

# Process each image in the test directory
for idx, file_name in enumerate(test_images, start=1):
    image_path = os.path.join(test_dir, file_name)
    image = Image.open(image_path)

    # Apply the transformation to the image
    image = transform_test(image).unsqueeze(0).to(device)  # Add batch dimension

    # Make the prediction
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    # 1 = dog, 0 = cat
    label = predicted.item()

    # Append result in the format (id, label)
    results.append([idx, label])



/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_24/1038562633.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

In [4]:
# Create a DataFrame and save to CSV
results_df = pd.DataFrame(results, columns=['id', 'label'])
results_df.to_csv('/kaggle/working/submission.csv', index=False)

# Print the first few results to check
print(results_df.head())


   id  label
0   1      0
1   2      1
2   3      1
3   4      1
4   5      1
